In [ ]:
import pandas as pd
import subprocess, json, datetime

In [2]:
# Rodar o comando docker ps -a e capturar todos os nomes dos containers
def get_containers():
    command = "docker ps -a --format '{{.Names}}'"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    return stdout.decode().split('\n')[:-1]

In [13]:
# Encontrar o arquivo companies.csv
try:
    df = pd.read_csv('../airflow/dags/data/companies.csv')
except FileNotFoundError:
    print('Arquivo companies.csv não encontrado')
    exit(1)

In [ ]:
empresas = ["CMIG", "ITSA", 'NEOE']
#empresas = df['issuingCompany'].unique() # Pegar todas as empresas do arquivo companies.csv
# Lista de containers que serão utilizados para rodar o script
# containers = get_containers()

In [19]:
df['dateListing'] = pd.to_datetime(df['dateListing'], errors='coerce')
df['dateListing'].fillna(pd.Timestamp('2010-01-01'), inplace=True)
df['dateListing'] = df['dateListing'].dt.year
lista = df[df['issuingCompany'].isin(empresas)][['issuingCompany', 'dateListing']]
lista['dateListing'] = lista['dateListing'].astype(int).apply(lambda x: 2020 if x < 2011 else x)

In [11]:
def execute_on_container(container, company, year):
    # Cria um dicionário com as variáveis
    variables_dict = {
        "b3_ano": year,
        "b3_empresa": company
    }
    variables = json.dumps(variables_dict)

    # Comando Airflow
    airflow_command = ["docker", "exec", container, "airflow", "dags", "trigger", "-c", variables, "b3_data_extraction"]


    print(f"Executando comando no container {container}")
    print(f"Comando: {' '.join(airflow_command)}")

    try:
        result = subprocess.run(airflow_command, check=True, capture_output=True, text=True)
        print(f"Saída do comando:\n{result.stdout}")
        if result.stderr:
            print(f"Erro:\n{result.stderr}")
        return result
    except subprocess.CalledProcessError as e:
        print(f"Erro ao executar o comando no container {container}:\n{e.stderr}")
        return None

In [21]:
ano_atual = datetime.datetime.now().year + 1
for index, row in lista.iterrows():
    company = row['issuingCompany']
    min_year = row['dateListing']
    for year in range(min_year, ano_atual):
        execute_on_container("airflow-airflow-webserver-1", company, str(year))

Executando comando no container airflow-airflow-webserver-1
Comando: docker exec airflow-airflow-webserver-1 airflow dags trigger -c {"b3_ano": "2020", "b3_empresa": "CMIG"} b3_data_extraction
Saída do comando:
[2025-03-14T02:48:53.277+0000] {__init__.py:43} INFO - Loaded API auth backend: airflow.api.auth.backend.basic_auth
[2025-03-14T02:48:53.278+0000] {__init__.py:43} INFO - Loaded API auth backend: airflow.api.auth.backend.session
                   |                   |                   | data_interval_sta |                   |          |                  | last_scheduling_ |                   |          |            |       
conf               | dag_id            | dag_run_id        | rt                | data_interval_end | end_date | external_trigger | decision         | logical_date      | run_type | start_date | state 
===================+===================+===================+===================+===================+==========+==================+==================+=========